In [23]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import json
import pandas as pd
from loguru import logger
import csv
import re

from OpenBookkeeping.sql_db import query_by_col, update_by_col, add_detail, del_by_col, query_table, query_by_str

from OpenBookkeeping.gloab_info import prop_type_items, liability_currency_types

In [10]:
db_path = "E:\\nextcloud\\use.bk"

query_str = """
        select type, name, start_date, term_month, rate, currency, ctype, sum(amount) from prop 
        LEFT outer join prop_details
        on prop.id = prop_details.target_id group by name;
        """

In [12]:
props = query_by_str(db_path, query_str)

2024-01-22 16:56:40.537 | DEBUG    | OpenBookkeeping.sql_db:query_by_str:137 - sql_str='\n        select type, name, start_date, term_month, rate, currency, ctype, sum(amount) from prop \n        LEFT outer join prop_details\n        on prop.id = prop_details.target_id group by name;\n        '


In [24]:
prop_df = pd.DataFrame(props, columns=['type', 'name', 'start_date', 'term_month', 'rate', 'currency', 'ctype', 'sum_amount'])
prop_df.fillna(0, inplace=True)
prop_df['type_cn'] = prop_df['type'].apply(lambda x: prop_type_items[x])

In [41]:
groups = prop_df[['name','type_cn', 'sum_amount']].groupby('type_cn')
for g in groups:
    print(g)

('固定资产',     name type_cn  sum_amount
0  2B801    固定资产   5000000.0
1  42001    固定资产   7500000.0
5   何-工资    固定资产         0.0
6   姜-工资    固定资产         0.0
8   对外债权    固定资产     15600.0)
('流动资产',        name type_cn  sum_amount
4     中行-活期    流动资产       105.0
11    建行-活期    流动资产      6609.0
15  建行对公-活期    流动资产      2580.0
17    招行-活期    流动资产      5255.0
19    支付宝基金    流动资产     10370.0)
('短期负债',         name type_cn  sum_amount
2     jy-园丁贷    短期负债         0.0
3   jy-建行消费贷    短期负债         0.0
10    建行-抵押贷    短期负债   1560000.0
12    建行-消费贷    短期负债         0.0
16    招行-信用卡    短期负债         0.0
18    招行-消费贷    短期负债    200000.0
20    浦发-信用卡    短期负债         0.0)
('长期负债',          name type_cn  sum_amount
7       姜家备用金    长期负债         0.0
9    工行2001房贷    长期负债   2530000.0
13  建行801公积金贷    长期负债    400000.0
14    建行801房贷    长期负债    616551.0
21        生活费    长期负债         0.0)


In [33]:
as_amount = np.sum(prop_df[prop_df['type'] <= 1]['sum_amount'])
de_amount = np.sum(prop_df[prop_df['type'] >= 2]['sum_amount'])
ne_amount = as_amount - de_amount
print(f'资产:{as_amount/10000}万, 负债{de_amount/10000}万，净值{ne_amount/10000}万')

资产:1254.0519万, 负债530.6551万，净值723.3968万


In [36]:
as_df = prop_df[prop_df['type'] <= 1]
de_df = prop_df[prop_df['type'] >= 2]